In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [6]:
street_map = gpd.read_file('data/Washington_DC_Boundary.shp')
fig, ax = plt.subplots(figsize=(15,15))
street_map.plot(ax=ax)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "fiona/ogrext.pyx", line 136, in fiona.ogrext.gdal_open_vector
  File "fiona/_err.pyx", line 291, in fiona._err.exc_wrap_pointer
fiona._err.CPLE_OpenFailedError: Unable to open data/Washington_DC_Boundary.shx or data/Washington_DC_Boundary.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4526/1042891753.py", line 1, in <module>
    street_map = gpd.read_file('data/Washington_DC_Boundary.shp')
  File "/home/codespace/.python/current/lib/python3.10/site-packages/geopandas/io/file.py", line 259, in _read_file
    return _read_file_fiona(
  File "/home/codespace/.python/current/lib/python3.10/site-packages/geopandas/io/file.py", l

In [5]:
df = pd.read_csv('data/good_data/Albuquerque_Final_2022-06-18.csv')
crs = {'init':'epsg:4326'}


# keep only common_name, condition, latitude_coordinate, longitude_coordinate, and native columns
df = df[['common_name', 'condition', 'latitude_coordinate', 'longitude_coordinate', 'native']]
# convert condition to numerical
df['condition'] = df['condition'].replace({'excellent': 4, 'good': 3, 'fair': 2, 'poor': 1, 'dead/dying': 0, 'dead': 0})
# one hot "common_name" column
df = pd.get_dummies(df, columns=["common_name"])
# one hot native column
df = pd.get_dummies(df, columns=["native"])
# drop native_no_info column
df.drop(columns=['native_no_info'], inplace=True)
# drop rows where condition is null
df = df.dropna(subset=['condition'])
df.head()

,condition,latitude_coordinate,longitude_coordinate,common_name_Afghan pine,common_name_American basswood,common_name_American elm,common_name_American holly,common_name_Amur maple,common_name_Apple,common_name_Apricot,...,common_name_Siberian elm,common_name_Silver maple,common_name_Spruce,common_name_Texas red oak,common_name_Tree of heaven,common_name_Velvet ash arizona ash,common_name_Vitex,common_name_White ash,native_introduced,native_naturally_occurring
0,0.0,35.060456,-106.568366,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,35.052176,-106.581127,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,35.052450,-106.575056,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,3.0,35.149632,-106.716539,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,3.0,35.149744,-106.716640,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
X = df.drop('condition', axis=1)
y = df['condition']

In [ ]:
geometry = [Point(xy) for xy in zip(df['longitude_coordinate'], df['latitude_coordinate'])]
geo_df = gpd.GeoDataFrame(df, #specify our data
                          crs=crs #specify our coordinate reference system
                          geometry=geometry) #specify the geometry list we created
geo_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
street_map.plot(ax=ax, alpha=0.4, color='grey')
geo_df[geo_df['WnvPresent'] == 0].plot(ax=ax, 
                                       markersize=20, 
                                       color='blue', 
                                       marker='o', 
                                       label='Neg')
geo_df[geo_df['WnvPresent'] == 1].plot(ax=ax, 
                                       markersize=20, 
                                       color='red', 
                                       marker='^', 
                                       label='Pos')
plt.legend(prop={'size':15})